In [8]:
%pip install opencv-python
%pip install pillow
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install tensorflow==2.10
%pip install keras

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Importing all necessary libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

img_width, img_height = 224, 224


2022-10-09 21:25:27.349391: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-09 21:25:27.914641: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib
2022-10-09 21:25:27.914726: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-09 21:25:27.991771: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-09 21:25:29.689921: W 

In [2]:
train_data_dir = '/home/subhrajit/Desktop/EE604/Assignment 2/datathonindoml-2022/train/train'
validation_data_dir = '/home/subhrajit/Desktop/EE604/Assignment 2/datathonindoml-2022/validation/validation'
nb_train_samples =16000
nb_validation_samples = 900
epochs = 10
batch_size = 16

In [3]:
if K.image_data_format() == 'channels_first':
	input_shape = (3, img_width, img_height)
else:
	input_shape = (img_width, img_height, 3)


In [4]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

2022-10-09 22:15:38.921268: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-09 22:15:38.921309: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pavillion): /proc/driver/nvidia/version does not exist
2022-10-09 22:15:38.922262: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model.compile(loss='binary_crossentropy',
			optimizer='rmsprop',
			metrics=['accuracy'])


In [6]:
train_datagen = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
	train_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

model.fit_generator(
	train_generator,
	steps_per_epoch=nb_train_samples // batch_size,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=nb_validation_samples // batch_size)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


/tmp/ipykernel_4868/3762154218.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [7]:
model.save_weights('model_saved.h5')


In [8]:
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import numpy as np

from keras.models import load_model

model = load_model('model_saved.h5')

image = load_img('v_data/test/planes/5.jpg', target_size=(224, 224))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,224,224,3)
label = model.predict(img)
print("Predicted Class (0 - Cars , 1- Planes): ", label[0][0])


ImportError: cannot import name 'load_img' from 'keras.preprocessing.image' (/home/subhrajit/.local/lib/python3.8/site-packages/keras/preprocessing/image.py)

In [ ]:

import ast
import sys

import cv2
import matplotlib.pyplot as plt
import pandas as pd
import PIL.Image
import torch
import torch.nn.functional as F

sys.path.insert(0, '..')
from pytorch_image_classification import (
    get_default_config,
    create_model,
    create_transform,
)
config = get_default_config()
config.merge_from_file('../configs/imagenet/resnet18.yaml')
device = torch.device(config.device)

model = create_model(config)
checkpoint = torch.load(
    '../experiments/imagenet/resnet18/exp00/checkpoint_00090.pth')

model.load_state_dict(checkpoint['model'])
model.to(device)
_ = model.eval()
transform = create_transform(config, is_train=False)
%%capture
!wget https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt
with open('imagenet1000_clsidx_to_labels.txt') as f:
    index2label = ast.literal_eval(f.read())
%%capture
!wget https://images.pexels.com/photos/2071873/pexels-photo-2071873.jpeg
image = cv2.imread('pexels-photo-2071873.jpeg')
plt.imshow(image[:, :, ::-1])
plt.show()

data = transform(PIL.Image.fromarray(image))
with torch.no_grad():
    pred = model(data.unsqueeze(0).to(device))
prob = F.softmax(pred, dim=1).cpu()

scores, indices = prob.topk(k=5)
scores = scores.numpy().ravel()

indices = indices.numpy().ravel()
names = [index2label[index] for index in indices]
pd.DataFrame({'label': names, 'score': scores})
label	score
0	Egyptian cat	0.836771
1	lynx, catamount	0.022285
2	tabby, tabby cat	0.020165
3	Siamese cat, Siamese	0.017015
4	tiger cat	0.007150
 